In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [3]:
df = pd.read_csv('SZFZ_2.csv')
df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df['DateConfirm'] = pd.to_datetime(df['DateConfirm'])
df.to_csv('SZFZ_working_monthly_2.csv',index=None)

In [3]:
df.drop('DateConfirm',axis=1,inplace=True)
df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])

df['DateSymptom'] = df['DateSymptom'].dt.to_period('M')

date_start = df.sort_values(by='DateSymptom').iloc[0]['DateSymptom']
date_end = df.sort_values(by='DateSymptom').iloc[-1]['DateSymptom']
date_index = pd.PeriodIndex(start=date_start,end=date_end,freq='m')

df.loc[df['Class']=='SZFZ','Class'] = 0
df.loc[:,'Class']=df.loc[:,'Class'].astype(float)

TYPES = ['I_new','I','A(not tp)','tp','A','I/(I+A)','I/population']
CLASSES = df['Class'].unique()
CLASSES.sort()
CLASSES = CLASSES[::-1]
CLASSES = np.append(CLASSES,[1,])
CLASSES = CLASSES.astype(float)

POPULATION = [1118/18,] * (len(CLASSES)-1)
POPULATION = np.append(POPULATION,[1118,])

type_class = [(t,c)for t in TYPES for c in CLASSES]

type_index = pd.MultiIndex.from_tuples(type_class)

org_df = pd.DataFrame(index=date_index,columns=type_index)

org_df.loc[:,['I_new','I']] = 0

date_length = len(date_index)
for c in CLASSES[:-1]:
    class_df = df[df['Class']==c]
    t = 'I'
    n = 1
    cur_df = class_df[class_df['Type']==n]['DateSymptom'].value_counts().sort_index()
    cur_dates = cur_df.index.values
    cur_vals = cur_df.values
    l = 0
    for i in range(date_length):
        if i>0:
            org_df.loc[date_index[i],(t,c)] = org_df.loc[date_index[i-1],(t,c)] * 5/6
        if l<len(cur_dates) and date_index[i]==cur_dates[l]:
            org_df.loc[date_index[i],(t,c)] += cur_vals[l]
            org_df.loc[date_index[i],('I_new',c)] = cur_vals[l]
            l += 1

for t in ['I_new','I']:
    org_df.loc[:,(t,1)] = org_df.loc[:,t].iloc[:,:-1].sum(axis=1)

I_total = 0
for c,p in zip(CLASSES[:-1],POPULATION[:-1]):
    I = len(df.loc[np.logical_and(df['Class']==c,df['Type']==1)]) 
    I_total += I
    A_ntp = len(df.loc[np.logical_and(df['Class']==c,df['Type']==2)]) 
    tp = len(df.loc[np.logical_and(df['Class']==c,df['Type']==3)]) 
    org_df.loc[date_index[0],('A(not tp)',c)] = A_ntp
    org_df.loc[date_index[0],('tp',c)] = tp
    org_df.loc[date_index[0],('A',c)] = A_ntp + tp
    org_df.loc[date_index[0],('I/(I+A)',c)] = I/(I+A_ntp + tp)
    org_df.loc[date_index[0],('I/population',c)] = I/p

for t in ['A(not tp)','tp','A']:
    org_df.loc[date_index[0],(t,1)] = org_df.loc[date_index[0],t].iloc[:-1].sum()
    
org_df.loc[date_index[0],('I/(I+A)',1)] = I_total/(I_total+org_df.loc[date_index[0],('A',1)])
org_df.loc[date_index[0],('I/population',1)] = I_total/POPULATION[-1]

org_df=org_df.swaplevel(axis=1)
v = pd.Categorical(org_df.columns.get_level_values(0), 
                   categories=CLASSES, 
                   ordered=True)
v2 = pd.Categorical(org_df.columns.get_level_values(1), 
                    categories=TYPES,
                    ordered=True)
org_df.columns = pd.MultiIndex.from_arrays([v, v2]) 
org_df = org_df.sort_index(axis=1, level=[0, 1])
org_df

52.0                                                51.0       \
        I_new    I A(not tp)   tp    A I/(I+A) I/population I_new    I   
2013-12     0  0.0         0    0    0       1    0.0161002     0  0.0   
2014-01     0  0.0       NaN  NaN  NaN     NaN          NaN     0  0.0   
2014-02     0  0.0       NaN  NaN  NaN     NaN          NaN     0  0.0   
2014-03     0  0.0       NaN  NaN  NaN     NaN          NaN     0  0.0   
2014-04     0  0.0       NaN  NaN  NaN     NaN          NaN     0  0.0   
2014-05     1  1.0       NaN  NaN  NaN     NaN          NaN     1  1.0   

                      ...      0.0                        1.0              \
        A(not tp)     ...         A I/(I+A) I/population I_new          I   
2013-12         0     ...         0       1     0.193202     1   1.000000   
2014-01       NaN     ...       NaN     NaN          NaN     0   0.833333   
2014-02       NaN     ...       NaN     NaN          NaN     3   3.694444   
2014-03       NaN     ...       NaN     NaN          NaN     3   6.078704   
2014-04       NaN     ...       NaN     NaN          NaN    23  28.065586   
2014-05       NaN     ...       NaN     NaN          NaN    36  59.387989   

                                                    
        A(not tp)   tp    A   I/(I+A) I/population  
2013-12         0    3    3  0.956522     0.059034  
2014-01       NaN  NaN  NaN       NaN          NaN  
2014-02       NaN  NaN  NaN       NaN          NaN  
2014-03       NaN  NaN  NaN       NaN          NaN  
2014-04       NaN  NaN  NaN       NaN          NaN  
2014-05       NaN  NaN  NaN       NaN          NaN  

[6 rows x 98 columns]

In [4]:
org_df.to_csv('SZFZ_monthly_ver2.csv')

In [2]:
df = pd.read_csv('SYZX_2.csv')
df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df['DateConfirm'] = pd.to_datetime(df['DateConfirm'])
df.to_csv('SYZX_working_monthly_2.csv',index=None)

In [3]:
df.drop('DateConfirm',axis=1,inplace=True)
df.drop(94,inplace=True)

df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df['DateSymptom'] = df['DateSymptom'].dt.to_period('M')

df.loc[df['Class'].isnull(),'Class'] = 0

date_start = df[df['Type']!=2].sort_values(by='DateSymptom').iloc[0]['DateSymptom']
date_end = df[df['Type']!=2].sort_values(by='DateSymptom').iloc[-1]['DateSymptom']
date_index = pd.PeriodIndex(start=date_start,end=date_end,freq='m')

df.loc[:,'Class']=df.loc[:,'Class'].astype(float)

TYPES = ['I_new','I','A(not tp)','tp','A','I/(I+A)','I/population']
CLASSES = df['Class'].unique()
CLASSES.sort()
CLASSES = CLASSES[::-1]
CLASSES = np.append(CLASSES,[1,])
CLASSES = CLASSES.astype(float)

POPULATION = [2760/30,] * len(CLASSES[:-1])
POPULATION = np.append(POPULATION,[3858,])

type_class = [(t,c)for t in TYPES for c in CLASSES]

type_index = pd.MultiIndex.from_tuples(type_class)

org_df = pd.DataFrame(index=date_index,columns=type_index)

org_df.loc[:,['I_new','I']] = 0

date_length = len(date_index)
for c in CLASSES[:-1]:
    class_df = df[df['Class']==c]
    t = 'I'
    n = 1
    cur_df = class_df[class_df['Type']==n]['DateSymptom'].value_counts().sort_index()
    cur_dates = cur_df.index.values
    cur_vals = cur_df.values
    l = 0
    for i in range(date_length):
        if i>0:
            org_df.loc[date_index[i],(t,c)] = org_df.loc[date_index[i-1],(t,c)] * 5/6
        if l<len(cur_dates) and date_index[i]==cur_dates[l]:
            org_df.loc[date_index[i],(t,c)] += cur_vals[l]
            org_df.loc[date_index[i],('I_new',c)] = cur_vals[l]
            l += 1

for t in ['I_new','I']:
    org_df.loc[:,(t,1)] = org_df.loc[:,t].iloc[:,:-1].sum(axis=1)

I_total = 0
for c,p in zip(CLASSES[:-1],POPULATION[:-1]):
    I = len(df.loc[np.logical_and(df['Class']==c,df['Type']==1)]) 
    I_total += I
    A_ntp = len(df.loc[np.logical_and(df['Class']==c,df['Type']==2)]) 
    tp = len(df.loc[np.logical_and(df['Class']==c,df['Type']==3)]) 
    org_df.loc[date_index[0],('A(not tp)',c)] = A_ntp
    org_df.loc[date_index[0],('tp',c)] = tp
    org_df.loc[date_index[0],('A',c)] = A_ntp + tp
    org_df.loc[date_index[0],('I/(I+A)',c)] = I/(I+A_ntp + tp)
    org_df.loc[date_index[0],('I/population',c)] = I/p

for t in ['A(not tp)','tp','A']:
    org_df.loc[date_index[0],(t,1)] = org_df.loc[date_index[0],t].iloc[:-1].sum()
    
org_df.loc[date_index[0],('I/(I+A)',1)] = I_total/(I_total+org_df.loc[date_index[0],('A',1)])
org_df.loc[date_index[0],('I/population',1)] = I_total/POPULATION[-1]

org_df=org_df.swaplevel(axis=1)
v = pd.Categorical(org_df.columns.get_level_values(0), 
                   categories=CLASSES, 
                   ordered=True)
v2 = pd.Categorical(org_df.columns.get_level_values(1), 
                    categories=TYPES,
                    ordered=True)
org_df.columns = pd.MultiIndex.from_arrays([v, v2]) 
org_df = org_df.sort_index(axis=1, level=[0, 1])
org_df

104.0                                                    102.0  \
        I_new         I A(not tp)   tp    A I/(I+A) I/population I_new   
2013-03     0  0.000000         3    0    3     0.4    0.0217391     0   
2013-04     0  0.000000       NaN  NaN  NaN     NaN          NaN     0   
2013-05     0  0.000000       NaN  NaN  NaN     NaN          NaN     0   
2013-06     0  0.000000       NaN  NaN  NaN     NaN          NaN     0   
2013-07     0  0.000000       NaN  NaN  NaN     NaN          NaN     0   
2013-08     2  2.000000       NaN  NaN  NaN     NaN          NaN     0   
2013-09     0  1.666667       NaN  NaN  NaN     NaN          NaN     0   
2013-10     0  1.388889       NaN  NaN  NaN     NaN          NaN     1   
2013-11     0  1.157407       NaN  NaN  NaN     NaN          NaN     0   

                                ...      0.0                          1.0    \
                I A(not tp)     ...          A   I/(I+A) I/population I_new   
2013-03  0.000000         1     ...          1  0.666667    0.0217391     1   
2013-04  0.000000       NaN     ...        NaN       NaN          NaN     1   
2013-05  0.000000       NaN     ...        NaN       NaN          NaN     1   
2013-06  0.000000       NaN     ...        NaN       NaN          NaN    10   
2013-07  0.000000       NaN     ...        NaN       NaN          NaN     9   
2013-08  0.000000       NaN     ...        NaN       NaN          NaN    18   
2013-09  0.000000       NaN     ...        NaN       NaN          NaN    17   
2013-10  1.000000       NaN     ...        NaN       NaN          NaN    17   
2013-11  0.833333       NaN     ...        NaN       NaN          NaN     2   

                                                             
                 I A(not tp)   tp    A I/(I+A) I/population  
2013-03   1.000000        17    2   19     0.8    0.0196993  
2013-04   1.833333       NaN  NaN  NaN     NaN          NaN  
2013-05   2.527778       NaN  NaN  NaN     NaN          NaN  
2013-06  12.106481       NaN  NaN  NaN     NaN          NaN  
2013-07  19.088735       NaN  NaN  NaN     NaN          NaN  
2013-08  33.907279       NaN  NaN  NaN     NaN          NaN  
2013-09  45.256066       NaN  NaN  NaN     NaN          NaN  
2013-10  54.713388       NaN  NaN  NaN     NaN          NaN  
2013-11  47.594490       NaN  NaN  NaN     NaN          NaN  

[9 rows x 98 columns]

In [4]:
org_df.to_csv('SYZX_monthly_ver2.csv')

In [5]:
df = pd.read_csv('MBZX_2.csv')
df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df['DateConfirm'] = pd.to_datetime(df['DateConfirm'])
df.to_csv('MBZX_working_monthly_2.csv',index=None)

In [9]:
df.drop('DateConfirm',axis=1,inplace=True)
df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])

df['DateSymptom'] = df['DateSymptom'].dt.to_period('M')

date_start = df.sort_values(by='DateSymptom').iloc[0]['DateSymptom']
date_end = df.sort_values(by='DateSymptom').iloc[-1]['DateSymptom']
date_index = pd.PeriodIndex(start=date_start,end=date_end,freq='m')

df.loc[:,'Class']=df.loc[:,'Class'].astype(float)

TYPES = ['I_new','I','A(not tp)','tp','A','I/(I+A)','I/population']
CLASSES = df['Class'].unique()
CLASSES.sort()
CLASSES = CLASSES[::-1]
CLASSES = np.append(CLASSES,[1,])
CLASSES = CLASSES.astype(float)

POPULATION = [47,68,303]

type_class = [(t,c)for t in TYPES for c in CLASSES]

type_index = pd.MultiIndex.from_tuples(type_class)

org_df = pd.DataFrame(index=date_index,columns=type_index)

org_df.loc[:,['I_new','I']] = 0

date_length = len(date_index)
for c in CLASSES[:-1]:
    class_df = df[df['Class']==c]
    t = 'I'
    n = 1
    cur_df = class_df[class_df['Type']==n]['DateSymptom'].value_counts().sort_index()
    cur_dates = cur_df.index.values
    cur_vals = cur_df.values
    l = 0
    for i in range(date_length):
        if i>0:
            org_df.loc[date_index[i],(t,c)] = org_df.loc[date_index[i-1],(t,c)] * 5/6
        if l<len(cur_dates) and date_index[i]==cur_dates[l]:
            org_df.loc[date_index[i],(t,c)] += cur_vals[l]
            org_df.loc[date_index[i],('I_new',c)] = cur_vals[l]
            l += 1

for t in ['I_new','I']:
    org_df.loc[:,(t,1)] = org_df.loc[:,t].iloc[:,:-1].sum(axis=1)

I_total = 0
for c,p in zip(CLASSES[:-1],POPULATION[:-1]):
    I = len(df.loc[np.logical_and(df['Class']==c,df['Type']==1)]) 
    I_total += I
    A_ntp = len(df.loc[np.logical_and(df['Class']==c,df['Type']==2)]) 
    tp = len(df.loc[np.logical_and(df['Class']==c,df['Type']==3)]) 
    org_df.loc[date_index[0],('A(not tp)',c)] = A_ntp
    org_df.loc[date_index[0],('tp',c)] = tp
    org_df.loc[date_index[0],('A',c)] = A_ntp + tp
    org_df.loc[date_index[0],('I/(I+A)',c)] = I/(I+A_ntp + tp)
    org_df.loc[date_index[0],('I/population',c)] = I/p

for t in ['A(not tp)','tp','A']:
    org_df.loc[date_index[0],(t,1)] = org_df.loc[date_index[0],t].iloc[:-1].sum()
    
org_df.loc[date_index[0],('I/(I+A)',1)] = I_total/(I_total+org_df.loc[date_index[0],('A',1)])
org_df.loc[date_index[0],('I/population',1)] = I_total/POPULATION[-1]

org_df=org_df.swaplevel(axis=1)
v = pd.Categorical(org_df.columns.get_level_values(0), 
                   categories=CLASSES, 
                   ordered=True)
v2 = pd.Categorical(org_df.columns.get_level_values(1), 
                    categories=TYPES,
                    ordered=True)
org_df.columns = pd.MultiIndex.from_arrays([v, v2]) 
org_df = org_df.sort_index(axis=1, level=[0, 1])
org_df

8.0                                                        7.0       \
        I_new         I A(not tp)   tp    A   I/(I+A) I/population I_new    I   
2014-03     0  0.000000         1    3    4  0.692308     0.191489     0  0.0   
2014-04     1  1.000000       NaN  NaN  NaN       NaN          NaN     0  0.0   
2014-05     0  0.833333       NaN  NaN  NaN       NaN          NaN     0  0.0   
2014-06     1  1.694444       NaN  NaN  NaN       NaN          NaN     0  0.0   
2014-07     1  2.412037       NaN  NaN  NaN       NaN          NaN     0  0.0   
2014-08     1  3.010031       NaN  NaN  NaN       NaN          NaN     0  0.0   
2014-09     5  7.508359       NaN  NaN  NaN       NaN          NaN     1  1.0   

                      ...                                  1.0            \
        A(not tp)     ...         A I/(I+A) I/population I_new         I   
2014-03         0     ...         0       1    0.0147059     0  0.000000   
2014-04       NaN     ...       NaN     NaN          NaN     1  1.000000   
2014-05       NaN     ...       NaN     NaN          NaN     0  0.833333   
2014-06       NaN     ...       NaN     NaN          NaN     1  1.694444   
2014-07       NaN     ...       NaN     NaN          NaN     1  2.412037   
2014-08       NaN     ...       NaN     NaN          NaN     1  3.010031   
2014-09       NaN     ...       NaN     NaN          NaN     6  8.508359   

                                                    
        A(not tp)   tp    A   I/(I+A) I/population  
2014-03         1    3    4  0.714286    0.0330033  
2014-04       NaN  NaN  NaN       NaN          NaN  
2014-05       NaN  NaN  NaN       NaN          NaN  
2014-06       NaN  NaN  NaN       NaN          NaN  
2014-07       NaN  NaN  NaN       NaN          NaN  
2014-08       NaN  NaN  NaN       NaN          NaN  
2014-09       NaN  NaN  NaN       NaN          NaN  

[7 rows x 21 columns]

In [10]:
org_df.to_csv('MBZX_monthly_ver2.csv')

In [6]:
df = pd.read_csv('QJYS_2.csv')
df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df['DateConfirm'] = pd.to_datetime(df['DateConfirm'])
df.to_csv('QJYS_working_monthly_2.csv',index=None)

In [12]:
df.drop('DateConfirm',axis=1,inplace=True)

df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

df['DateSymptom'] = pd.to_datetime(df['DateSymptom'])
df['DateSymptom'] = df['DateSymptom'].dt.to_period('M')

date_start = df[df['Type']!=2].sort_values(by='DateSymptom').iloc[0]['DateSymptom']
date_end = df[df['Type']!=2].sort_values(by='DateSymptom').iloc[-1]['DateSymptom']
date_index = pd.PeriodIndex(start=date_start,end=date_end,freq='m')

df.loc[:,'Class']=df.loc[:,'Class'].astype(float)

TYPES = ['I_new','I','A(not tp)','tp','A','I/(I+A)','I/population']
CLASSES = df['Class'].unique()
CLASSES.sort()
CLASSES = CLASSES[::-1]
CLASSES = np.append(CLASSES,[1,])
CLASSES = CLASSES.astype(float)

POPULATION = [1958/26,] * (len(CLASSES)-1)
POPULATION = np.append(POPULATION,[1958,])

type_class = [(t,c)for t in TYPES for c in CLASSES]

type_index = pd.MultiIndex.from_tuples(type_class)

org_df = pd.DataFrame(index=date_index,columns=type_index)

org_df.loc[:,['I_new','I']] = 0

date_length = len(date_index)
for c in CLASSES[:-1]:
    class_df = df[df['Class']==c]
    t = 'I'
    n = 1
    cur_df = class_df[class_df['Type']==n]['DateSymptom'].value_counts().sort_index()
    cur_dates = cur_df.index.values
    cur_vals = cur_df.values
    l = 0
    for i in range(date_length):
        if i>0:
            org_df.loc[date_index[i],(t,c)] = org_df.loc[date_index[i-1],(t,c)] * 5/6
        if l<len(cur_dates) and date_index[i]==cur_dates[l]:
            org_df.loc[date_index[i],(t,c)] += cur_vals[l]
            org_df.loc[date_index[i],('I_new',c)] = cur_vals[l]
            l += 1

for t in ['I_new','I']:
    org_df.loc[:,(t,1)] = org_df.loc[:,t].iloc[:,:-1].sum(axis=1)

I_total = 0
for c,p in zip(CLASSES[:-1],POPULATION[:-1]):
    I = len(df.loc[np.logical_and(df['Class']==c,df['Type']==1)]) 
    I_total += I
    A_ntp = len(df.loc[np.logical_and(df['Class']==c,df['Type']==2)]) 
    tp = len(df.loc[np.logical_and(df['Class']==c,df['Type']==3)]) 
    org_df.loc[date_index[0],('A(not tp)',c)] = A_ntp
    org_df.loc[date_index[0],('tp',c)] = tp
    org_df.loc[date_index[0],('A',c)] = A_ntp + tp
    org_df.loc[date_index[0],('I/(I+A)',c)] = I/(I+A_ntp + tp)
    org_df.loc[date_index[0],('I/population',c)] = I/p

for t in ['A(not tp)','tp','A']:
    org_df.loc[date_index[0],(t,1)] = org_df.loc[date_index[0],t].iloc[:-1].sum()
    
org_df.loc[date_index[0],('I/(I+A)',1)] = I_total/(I_total+org_df.loc[date_index[0],('A',1)])
org_df.loc[date_index[0],('I/population',1)] = I_total/POPULATION[-1]

org_df=org_df.swaplevel(axis=1)
v = pd.Categorical(org_df.columns.get_level_values(0), 
                   categories=CLASSES, 
                   ordered=True)
v2 = pd.Categorical(org_df.columns.get_level_values(1), 
                    categories=TYPES,
                    ordered=True)
org_df.columns = pd.MultiIndex.from_arrays([v, v2]) 
org_df = org_df.sort_index(axis=1, level=[0, 1])
org_df

230.0                                               223.0            \
        I_new    I A(not tp)   tp    A I/(I+A) I/population I_new         I   
2017-06     0  0.0         0    0    0       1    0.0132789     1  1.000000   
2017-07     0  0.0       NaN  NaN  NaN     NaN          NaN     0  0.833333   
2017-08     0  0.0       NaN  NaN  NaN     NaN          NaN     0  0.694444   
2017-09     0  0.0       NaN  NaN  NaN     NaN          NaN     0  0.578704   
2017-10     0  0.0       NaN  NaN  NaN     NaN          NaN     1  1.482253   
2017-11     0  0.0       NaN  NaN  NaN     NaN          NaN     1  2.235211   
2017-12     0  0.0       NaN  NaN  NaN     NaN          NaN     3  4.862676   
2018-01     1  1.0       NaN  NaN  NaN     NaN          NaN     0  4.052230   

                      ...                                1.0              \
        A(not tp)     ...         A I/(I+A) I/population I_new         I   
2017-06         2     ...         2    0.75    0.0796731     1  1.000000   
2017-07       NaN     ...       NaN     NaN          NaN     0  0.833333   
2017-08       NaN     ...       NaN     NaN          NaN     0  0.694444   
2017-09       NaN     ...       NaN     NaN          NaN     0  0.578704   
2017-10       NaN     ...       NaN     NaN          NaN     1  1.482253   
2017-11       NaN     ...       NaN     NaN          NaN     1  2.235211   
2017-12       NaN     ...       NaN     NaN          NaN     3  4.862676   
2018-01       NaN     ...       NaN     NaN          NaN     1  5.052230   

                                                    
        A(not tp)   tp    A   I/(I+A) I/population  
2017-06         2    0    2  0.777778   0.00357508  
2017-07       NaN  NaN  NaN       NaN          NaN  
2017-08       NaN  NaN  NaN       NaN          NaN  
2017-09       NaN  NaN  NaN       NaN          NaN  
2017-10       NaN  NaN  NaN       NaN          NaN  
2017-11       NaN  NaN  NaN       NaN          NaN  
2017-12       NaN  NaN  NaN       NaN          NaN  
2018-01       NaN  NaN  NaN       NaN          NaN  

[8 rows x 21 columns]

In [13]:
org_df.to_csv('QJYS_monthly_ver2.csv')